In [20]:
import os
import pandas as pd
print(f"pandas Ver.: {pd.__version__}")
import numpy as np
print(f"numpy  Ver.: {np.__version__}")
import cclib
print(f"cclib  Ver.: {cclib.__version__}")
from HessianTools import *

pandas Ver.: 2.2.3
numpy  Ver.: 1.26.4
cclib  Ver.: 1.8.1


In [26]:
# get all file names ending with .out

text_files = [
    f for f in os.listdir(os.getcwd()) 
    if f.endswith(".out")
]

dataframes = []

In [27]:
for filename in text_files:
    #load the output
    data = cclib.io.ccread(filename)

    #Create a list of 0 (carbons)
    mol_indexes = np.zeros(20)
    
    #Change the B(5) to -1 and N(7) to 1
    for i in range(20):
        j=0
        if data.atomnos[i] == 6:
            j+=1
        if data.atomnos[i] == 5:
            mol_indexes[i] = -1
        if data.atomnos[i] == 7:
            mol_indexes[i] = 1
            if j==20:
                print(filename)
    
    # Create a DataFrame for the current file and add to the list
    df = pd.DataFrame([mol_indexes], columns=[f'z{i}' for i in range(20)])

    # Add the HOMO and LUMO columns
    homo_value = data.moenergies[0][data.homos[0]]
    lumo_value = data.moenergies[0][data.homos[0] + 1]
    df["HOMO"] = homo_value
    df["LUMO"] = lumo_value

    # Add the SCF Energies column
    scfEnergy = data.scfenergies
    df["Energies"] = scfEnergy

    # Load the last gradient matrix and flatten it
    grad = data.grads[-1]
    grad_flat = data.grads.flatten()

    # Create the gradient column names
    grad_column = [f"grad[{i}][{j}]" for i in range(data.natom) for j in ['x','y','z']]
    grad_df = pd.DataFrame([grad_flat], columns=grad_column)
    df = pd.concat([df, grad_df], axis=1)    

    # Load the Hessian matrix and flatten it 
    hess = HessianTools(f"{filename[:-4]}.hess")
    hessian_flat = hess.hessian.flatten()
    
    # Create Hessian column names
    hessian_column = [f"hess[{i}][{j}]" for i in range(data.natom*3) for j in range(data.natom*3)]
    hessian_df = pd.DataFrame([hessian_flat], columns=hessian_column)
    df = pd.concat([df, hessian_df], axis=1)    
    
    df.index = [filename]  # Set the filename as the index
    dataframes.append(df)
    
all_data = pd.concat(dataframes)
    
    #df = df.append(pd.Series(mol_indexes, name=filename))
    #df.columns = [f'{i}' for i in range(20)]


In [28]:
all_data

,z0,z1,z2,z3,z4,z5,z6,z7,z8,z9,...,hess[110][101],hess[110][102],hess[110][103],hess[110][104],hess[110][105],hess[110][106],hess[110][107],hess[110][108],hess[110][109],hess[110][110]
index_246_carga_-2.out,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004612,1.460848e-08,-4.989482e-08,-0.000447,-2.869488e-08,1.192944e-08,0.003351,3.288611e-09,-9.029891e-07,0.090576
index_248_carga_-2.out,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002010,6.109221e-09,3.230895e-08,-0.004575,4.016259e-08,3.270608e-09,0.002196,3.509304e-07,-7.301644e-07,0.093377
index_o_carga_0.out,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004587,-3.109972e-10,1.645304e-09,-0.000508,-3.122193e-08,-5.326334e-09,0.004587,5.512057e-09,-9.844535e-07,0.088905
index_434_carga_0.out,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001682,1.182707e-08,-3.765515e-08,0.001854,-7.793978e-09,-2.037269e-08,0.004139,2.316809e-09,-1.116762e-06,0.016178
index_81_carga_2.out,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.003945,-5.356160e-09,-1.931375e-08,0.001096,2.242576e-08,-1.221837e-08,0.003945,8.911984e-10,-9.763140e-07,0.087532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
index_464_carga_0.out,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.004493,2.172942e-08,5.667942e-09,0.000685,-3.418302e-08,-3.623716e-09,0.003777,-9.843208e-09,-9.845170e-07,0.090814
index_55_carga_2.out,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004813,1.767993e-08,7.912118e-09,-0.001119,-2.096480e-08,2.577847e-08,0.002865,1.362232e-07,-9.602859e-07,0.083529
index_2_carga_1.out,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004157,1.114899e-08,-1.591869e-08,0.000847,-1.436836e-08,-9.294923e-09,0.003693,7.734467e-09,-9.812529e-07,0.090602
index_230_carga_-2.out,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001756,2.100714e-09,-5.086489e-08,0.001432,1.236412e-08,-3.583088e-09,0.003787,8.139841e-09,-9.241712e-07,0.085534


In [29]:
all_data.to_csv("output.csv", index=True)  # Use index=True to include the filename index in the CSV

In [25]:
print(data.moenergies[0][data.homos[0]])
print(data.moenergies[0][data.homos[0]+1])
print(data.atomnos)

mol_indexes = np.zeros(20)
for i in range(20):
    if data.atomnos[i] == 5:
        mol_indexes[i] = -1
    if data.atomnos[i] == 7:
        mol_indexes[i] = 1

print(mol_indexes)


-12.364907789490099
-10.50422049115615
[ 6  7  6  7  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6 29  1  1  1
  1  1  1  1  1  1  1  1  1  7  7  7  7]
[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [30]:
data = cclib.io.ccread("index_246_carga_-2.out")
data.natom

37

In [31]:
print(f"""gradiant shape: {data.grads.shape} 
Hessian  shape: {hess.hessian.shape}""")

gradiant shape: (1, 37, 3) 
Hessian  shape: (111, 111)


In [32]:
data.grads[-1]


array([[ 1.5656744e-02, -4.2801896e-02, -2.2000000e-08],
       [-5.2594477e-02, -4.5092647e-02,  1.5700000e-07],
       [-2.8747580e-03,  1.1426713e-02,  7.0000000e-09],
       [ 8.2848830e-03, -1.4474659e-02,  2.9000000e-08],
       [ 1.1696330e-02,  1.4071465e-02,  1.5000000e-08],
       [-1.2939195e-02,  1.1385443e-02,  1.3000000e-08],
       [-9.3580790e-03, -9.7866390e-03,  2.9000000e-08],
       [-4.7701280e-03,  2.4594820e-03,  1.0000000e-09],
       [-6.3510810e-03, -1.5852328e-02,  3.2000000e-08],
       [ 9.6213770e-03, -8.5175630e-03,  2.0000000e-09],
       [ 1.0050168e-02,  8.2269560e-03,  1.4000000e-08],
       [-6.4115290e-03,  1.6918814e-02,  2.1000000e-08],
       [-6.5642690e-03, -3.1061000e-03,  2.0000000e-09],
       [-9.0666210e-03,  6.0356880e-03,  4.7000000e-08],
       [-1.2560070e-02, -8.8433320e-03,  6.0000000e-09],
       [ 3.1816960e-03, -1.5629976e-02,  9.0000000e-09],
       [-5.2673945e-02,  3.1229144e-02,  9.2000000e-08],
       [ 6.9622680e-03,  1.4175

In [15]:
grad_flat = data.grads.flatten()
grad_flat

array([-9.5373000e-03, -1.0372917e-02, -1.0000000e-08,  5.0162550e-03,
        3.5990070e-03,  3.6000000e-08,  6.9272480e-03, -6.2956590e-03,
       -3.2000000e-08,  1.3468831e-02,  3.2943980e-03,  4.5000000e-08,
       -1.3584060e-03,  2.8819990e-03, -9.0000000e-09,  3.0658370e-03,
        4.3311660e-03, -1.2000000e-08, -9.0704840e-03,  1.2000500e-03,
        4.8000000e-08,  8.2452900e-04,  5.6304700e-04, -1.4000000e-08,
        1.2808890e-03, -9.0114340e-03,  5.3000000e-08,  3.8595510e-03,
        2.3753900e-03, -1.7000000e-08,  3.7605900e-03, -2.2196700e-03,
       -1.2000000e-08,  7.9681500e-04,  1.2653586e-02,  4.8000000e-08,
       -2.5277960e-03, -4.0759540e-03, -1.1000000e-08, -4.5296090e-03,
       -9.5414800e-04,  4.8000000e-08,  2.7778880e-03, -4.5695020e-03,
       -7.0000000e-09, -1.4653880e-03, -3.2516960e-03, -1.3000000e-08,
        1.9125938e-02, -3.1931700e-04,  4.2000000e-08, -5.7881710e-03,
        1.2812051e-02, -2.4000000e-08,  2.4481947e-02, -2.1364100e-04,
      

In [9]:
for i in range(data.natom*3):
    for j in range(data.natom*3):
        df[f'hess[{i}][{j}]'] = hess.hessian[i][j]